<center><h1>Hemnani_Hitika_HW3</h1></center>
<br>
<br>

Name: Hitika Hemnani
<br>
Github Username: hhemnani
<br>
USC ID: 8304678802

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import pandas as pd
import numpy as np
import os

Get the AReM Data Set

In [2]:
ARem_data = "../data/AReM"

### (b) Test and Train Data

In [3]:
###I manually Splitted Train and Test Data but it was difficult to access them during feature extraction so I am Appending Traing and Test data into list of directory/files
##So I can access them easily later on


training_data = []
testing_data = []

## looping through all the directories in the AReM Dataset
for activity in os.listdir(ARem_data):
    activity_path = os.path.join(ARem_data, activity)
    
    # For the images -->skip
    if not os.path.isdir(activity_path):
        continue
        
    # again looping through all the dataset files this time
    for file in os.listdir(activity_path):
        if file.endswith('.csv'):
            file_path = os.path.join(activity_path, file)
            
            ##Extracting File Numbers to split
            dataset_number = int(file.replace('dataset', '').replace('.csv', ''))

            ##Splitting the data
            if activity in ['bending1', 'bending2']:
                if dataset_number <= 2:
                    testing_data.append((file_path, activity))
                else:
                    training_data.append((file_path, activity))
            else:
                if dataset_number <= 3:
                    testing_data.append((file_path, activity))
                else:
                    training_data.append((file_path, activity))

In [4]:
#training_data
##testing_data

### (c) Feature Extraction

#### i. Research

ref:https://www.researchgate.net/figure/List-of-extracted-time-domain-features-from-each-window-size_tbl2_327523090


##### Common Time-Domain Features for Time Series Classification:
Minimum:
The smallest value in the time series. It helps capture the lower bound of the data range.

Maximum:
The largest value in the time series. It helps capture the upper bound of the data range.

Mean:
The average value of the time series. It provides information about the central tendency of the data.

Median:
The middle value when the time series data is sorted. The median is less sensitive to outliers than the mean.

Standard Deviation (STD):
A measure of the spread or dispersion of the values in the time series. It indicates how much variation or "noise" is present.

Variance:
The square of the standard deviation. It measures the degree of variation in the time series data.

Range:
The difference between the maximum and minimum values. It is another measure of data dispersion.

Root Mean Square (RMS):
The square root of the mean of the squared values. This can give insights into the overall energy of the signal.


#### ii. Extraction

In [5]:
##Feature Extraction function to extract time-domain features

def extracting_time_features(time_data):
    # minimum=np.min(time_data)
    # maximum=np.max(time_data)
    # mean=np.mean(time_data)
    # median=np.median(time_data)
    # STD=np.std(time_data)
    # Q1 = np.percentile(time_data, 25)
    # Q3 = np.percentile(time_data, 75)

    # return [minimum,maximum,mean,median,STD,Q1,Q3]
    ##this idea is not used much simpler to just add values directly to columns
    
    features = {}
    
    ###Here I was considering all the columns I was getting the wrong output as the time column was also considered 
    ##Hence skipping 1st column
    for i, column in enumerate(time_data.columns[1:], start=1):
        
        ##Error Handeling->type error occured
        series = pd.to_numeric(time_data[column], errors='coerce')
        series = series.dropna()
        if series.empty:
            continue
        
        features.update({
            f'min{i}': np.min(series),
            f'max{i}': np.max(series),
            f'mean{i}': np.mean(series),
            f'median{i}': np.median(series),
            f'std{i}': np.std(series),
            f'1st_quart{i}': np.percentile(series, 25),
            f'3rd_quart{i}': np.percentile(series, 75)
        })
    
    return features


In [6]:
training_df=[]
testing_df=[]

##looping to get all the datasets
for path, activity in training_data:
    time_data=pd.read_csv(path, sep=',', skiprows=4, on_bad_lines='skip')
    # print(time_data)
    features = extracting_time_features(time_data)
    features['activity'] = activity
    training_df.append(features)

##Same for testing data
for path, activity in testing_data:
    time_data = pd.read_csv(path, sep=',', skiprows=4, on_bad_lines='skip')
    features = extracting_time_features(time_data)
    features['activity'] = activity
    testing_df.append(features)

In [7]:
# Convert lists to DataFrames
training_df = pd.DataFrame(training_df)
testing_df = pd.DataFrame(testing_df)
new_dataset = pd.concat([training_df, testing_df], ignore_index=True)

# Adding 'Instance' column to match the assignment requirements
new_dataset.insert(0, 'Instance', range(1, len(new_dataset) + 1))
new_dataset.drop(columns=['activity'], inplace=True)

In [8]:
new_dataset

,Instance,min1,max1,mean1,median1,std1,1st_quart1,3rd_quart1,min2,max2,...,std5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std6,1st_quart6,3rd_quart6
0,1,35.00,47.40,43.954500,44.330,1.557210,43.00,45.0000,0.0,1.70,...,1.997520,35.3625,36.500,0.0,1.79,0.493292,0.430,0.512971,0.000,0.940
1,2,33.00,47.75,42.179812,43.500,3.666840,39.15,45.0000,0.0,3.00,...,3.845436,30.4575,36.330,0.0,2.18,0.613521,0.500,0.523771,0.000,1.000
2,3,33.00,45.75,41.678063,41.750,2.241152,41.33,42.7500,0.0,2.83,...,2.408514,28.4575,31.250,0.0,1.79,0.383292,0.430,0.388759,0.000,0.500
3,4,37.00,48.00,43.454958,43.250,1.384653,42.50,45.0000,0.0,1.58,...,2.486268,22.2500,24.000,0.0,5.26,0.679646,0.500,0.621885,0.430,0.870
4,5,36.25,48.00,43.969125,44.500,1.616677,43.31,44.6700,0.0,1.50,...,3.314843,20.5000,23.750,0.0,2.96,0.555312,0.490,0.487318,0.000,0.830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,35.50,46.25,43.174938,43.670,1.986979,42.50,44.5000,0.0,2.12,...,2.980866,12.7500,16.500,0.0,5.72,0.911979,0.830,0.665467,0.470,1.220
84,85,32.75,47.00,42.760562,44.500,3.395376,41.33,45.3725,0.0,3.34,...,4.292096,13.0000,18.565,0.0,5.73,0.842271,0.710,0.721413,0.430,1.090
85,86,19.33,43.50,34.227771,35.500,4.884480,30.50,37.7500,0.0,14.50,...,3.088871,14.7500,18.670,0.0,9.74,3.394125,3.100,1.790222,2.105,4.425
86,87,12.50,45.00,33.509729,34.125,4.845868,30.50,36.7500,0.0,13.05,...,3.130299,14.6275,18.750,0.0,8.96,3.378479,3.085,1.785497,2.060,4.440


#### iii. Standard Deviation

In [9]:
only_TDF = [col for col in new_dataset.columns if col not in ['Instance']]

##Bootstraping
bootstrap_no = 1000 
confidence_level = 0.90 
results_for_every_feature = {}

for feature in only_TDF:
    feature_data = new_dataset[feature]
    all_std = []
    for i in range(bootstrap_no):
        sampling = feature_data.sample(n=len(feature_data), replace=True)
        all_std.append(sampling.std())
    
    lower_bound= np.percentile(all_std, (1 - confidence_level) / 2 * 100)
    upper_bound = np.percentile(all_std, (1 + confidence_level) / 2 * 100)

    results_for_every_feature[feature] = {
        'std_dev': feature_data.std(),
        '90%_confidence_lower': lower_bound,
        '90%_confidence_upper': upper_bound 
    }

In [10]:
results_for_every_feature_df=pd.DataFrame(results_for_every_feature)

In [11]:
results_for_every_feature_df

,min1,max1,mean1,median1,std1,1st_quart1,3rd_quart1,min2,max2,mean2,...,std5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std6,1st_quart6,3rd_quart6
std_dev,9.624011,4.207745,5.276413,5.386624,1.769467,6.128143,5.031028,0.0,5.059656,1.577941,...,1.011324,6.122142,5.563553,0.046101,2.533515,1.157299,1.089858,0.516463,0.761645,1.527012
90%_confidence_lower,8.259495,3.182013,4.625094,4.745278,1.559068,5.533800,4.177165,0.0,4.619758,1.393645,...,0.793258,4.838210,4.409015,0.000000,2.261097,1.063831,1.005226,0.478211,0.693671,1.408581
90%_confidence_upper,10.760537,5.148286,5.834165,5.955424,1.953446,6.620401,5.739224,0.0,5.414007,1.711567,...,1.217155,7.284673,6.639889,0.078915,2.760399,1.218537,1.151525,0.543329,0.808547,1.599291


#### iv. Select Features

After analyzing the time-domain features extracted from the AReM dataset, I selected the following three features as the most important for human activity recognition:

Mean: It shows the average value of the data, which can help differentiate activities based on their overall data strength.

Standard Deviation: It measures how much the data varies over time, which is useful for telling apart activities with smooth or irregular patterns.

Maximum: It tells us the highest value of the data, which can help identify activities with sudden or intense movements.

## 2. ISLR 3.7.4

### (a) Linear Train

The cubic regression model has more parameters and can fit the training data more closely than the linear model, even if the true relationship is linear.

Therefore, the training RSS for the cubic regression will generally be lower than the training RSS for the linear regression.

### (b) Linear Test

Since the true relationship is linear, the linear regression model is the correct model.

The cubic regression model, being more flexible, may overfit the training data by capturing noise (random fluctuations) rather than the true underlying relationship.

### (c) Not Linear Train

The cubic regression model is more flexible and can better capture non-linear patterns in the data.

Therefore, the training RSS for the cubic regression will generally be lower than the training RSS for the linear regression.

### (d) Not Linear Testing

If the true relationship is non-linear, the linear regression model will underfit the data, resulting in poor performance on both training and test data.

The cubic regression model, being more flexible, can better capture the true non-linear relationship and generalize well to new data.